In [1]:
import pandas as pd
import yfinance as yf
import plotly.graph_objects as go
from datetime import timedelta
pd.set_option('display.max_rows',100)
pd.set_option('display.max_colwidth',1000)
pd.set_option('display.width',1000)
pd.set_option('display.max_columns',20)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))


## Cálculo do vol acumulado médio - periodo 60d, interval 5m

In [2]:
pd.set_option('display.max_rows',50)

def calc_volume(tick,period,interval):
    m5 = yf.Ticker(tick+".SA").history(period=period,interval=interval, prepost = True).reset_index()
    m5['ticker']=tick
    #trata horário de verão
    m5.loc[m5.Datetime.dt.date < pd.to_datetime("2021-02-22").date() , 'Datetime']  = (m5.Datetime - timedelta(minutes=60))
    m5.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    m5['Volume']=m5.Volume/100#1000000
    m5['vol_acumulado'] = m5.groupby(m5.Datetime.dt.date)['Volume'].cumsum()
    m5['hour']=m5.Datetime.dt.hour
    m5['minute']=m5.Datetime.dt.minute


    m5_media_qtd = pd.merge(
        m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].mean().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'vol_acum_medio'})
        ,m5.groupby([(m5.hour),(m5.minute)])['vol_acumulado'].count().to_frame().round(2).reset_index().rename(columns={'vol_acumulado':'qtd_vol_acum'})
        ,how='inner', on=['hour','minute']
             )

    m5_merge = pd.merge(m5,m5_media_qtd,how='left',on=['hour','minute'])
    m5_merge['pct_vol_acumulado'] = m5_merge.vol_acumulado / m5_merge.vol_acum_medio*100
    m5_merge=m5_merge.round(2)
    
    return m5_merge

In [13]:
%%time 
ibr=pd.read_csv('./ibr.txt')#.tail(2)

period="20d"
interval="5m"

for i, row in ibr.iterrows(): 
    if i == 0:
        df = calc_volume(row.ticker,period,interval)
    else:
        df = df.append(calc_volume(row.ticker,period,interval))

CPU times: user 1min 16s, sys: 3.07 s, total: 1min 19s
Wall time: 2min 11s


In [16]:
df[
    (       df.Datetime.dt.date == pd.to_datetime("today").date()  )
        &
    (  df.pct_vol_acumulado > 300      )
    & ( df.ticker != 'OMGE3')
].sort_values(by=['Datetime','ticker'], ascending=False)
#].sort_values(by=['pct_vol_acumulado'], ascending=False)

,Datetime,Open,High,Low,Close,Volume,ticker,vol_acumulado,hour,minute,vol_acum_medio,qtd_vol_acum,pct_vol_acumulado
1735,2021-03-08 15:50:57-03:00,11.49,11.49,11.49,11.49,0.0,SEER3,29029.0,15,50,8459.30,20,343.16
1734,2021-03-08 15:50:57-03:00,11.68,11.68,11.68,11.68,0.0,MTRE3,17828.0,15,50,5700.50,20,312.74
1727,2021-03-08 15:50:57-03:00,16.72,16.72,16.72,16.72,0.0,MRVE3,116420.0,15,50,36146.80,20,322.08
1711,2021-03-08 15:50:57-03:00,23.65,23.65,23.65,23.65,0.0,MGLU3,1210422.0,15,50,275283.65,20,439.70
1695,2021-03-08 15:50:54-03:00,23.86,23.86,23.86,23.86,0.0,LWSA3,271692.0,15,50,83198.25,20,326.56
1721,2021-03-08 15:50:50-03:00,23.06,23.06,23.06,23.06,0.0,LCAM3,83373.0,15,50,24542.70,20,339.71
1729,2021-03-08 15:50:46-03:00,8.85,8.85,8.85,8.85,0.0,PTBL3,133050.0,15,50,37603.20,20,353.83
1732,2021-03-08 15:50:46-03:00,30.33,30.33,30.33,30.33,0.0,LOGG3,33711.0,15,50,5157.50,20,653.63
1699,2021-03-08 15:50:46-03:00,27.04,27.04,27.04,27.04,0.0,ITUB4,1157208.0,15,50,356190.45,20,324.88
1711,2021-03-08 15:50:43-03:00,27.40,27.40,27.40,27.40,0.0,JBSS3,446042.0,15,50,129946.95,20,343.25


## Diário

In [20]:
%%time
def diario(tick, period):
    acao=tick+".SA"
    df = yf.Ticker(acao).history(period=period).reset_index()
    df['Acao'] = tick
    df['cotacao_hj'] = yf.Ticker(acao).history(period="1m").tail(1).Close.iloc[0]
    df['cotacao_dia_anterior'] = df['Close'].shift(1)
    df['pct_do_dia'] = (df.Close - df.cotacao_dia_anterior) / df.cotacao_dia_anterior*100
    df.drop(['Dividends', 'Stock Splits'], axis=1, inplace=True)
    df.Volume = df.Volume/1000000
    #df['media_vol'] = df.Volume.mean()
    #df['media_vol']= df.Volume.ewm(span=5, adjust=False).mean() #.astype(int) # MA
    df['media_vol']= df.Volume.rolling(window=5).mean() #EMA
    df['var_vol'] = df.Volume / df.media_vol.shift(2)
    df.loc[df['pct_do_dia'] >= 0, 'pct_longo'] = (df.cotacao_hj-df.Low)/df.cotacao_hj*100
    df.loc[df['pct_do_dia'] < 0, 'pct_longo'] = (df.High-df.cotacao_hj)/df.cotacao_hj*100

    return df

period="120d"
ibr=pd.read_csv('./ibr.txt')

# ibr = ibr[ibr.ticker == 'ELET6']
# ibr = ibr.reset_index(drop=True)

for i, row in ibr.iterrows():
    if i == 0:
        df = diario(row.ticker,period)
    else:
        df = df.append(diario(row.ticker,period))

CPU times: user 29.4 s, sys: 518 ms, total: 29.9 s
Wall time: 2min 14s


## Próximo do estouro

In [21]:
df=df[['Acao','Date','Open','High','Low','Close','cotacao_hj','cotacao_dia_anterior','pct_do_dia','media_vol','Volume','var_vol','pct_longo']]           
df = df.sort_values(by=['Acao','Date'],ascending=False).round(2)

df[
    #(df.pct_do_dia >= 4)
    (df.pct_do_dia.abs() >= 4)
   & (abs(df.pct_longo) <= 4)
   & (df.var_vol > 2)
   & (df.Date.dt.date <= pd.to_datetime('today').date() - timedelta(days=2) )
   #& (df.media_vol>0.5)
  ].sort_values(['Date','var_vol'],ascending=[False , False]).round(2)

,Acao,Date,Open,High,Low,Close,cotacao_hj,cotacao_dia_anterior,pct_do_dia,media_vol,Volume,var_vol,pct_longo
116,ALPK3,2021-03-03,7.62,7.62,7.29,7.29,7.40,7.60,-4.08,0.05,0.07,2.45,2.97
11,CSED3,2021-03-03,12.37,12.37,11.50,11.97,11.95,12.52,-4.39,0.78,1.16,2.17,3.51
94,BOAS3,2021-03-02,10.20,10.35,9.79,9.79,10.13,10.38,-5.68,2.66,6.43,4.88,2.17
115,ANIM3,2021-03-02,8.93,8.95,8.12,8.54,8.97,8.95,-4.58,4.90,9.75,3.63,-0.22
114,LAVV3,2021-03-01,7.06,7.07,6.50,6.70,6.95,7.20,-6.94,2.52,8.68,9.32,1.73
114,VLID3,2021-03-01,7.59,7.60,7.17,7.18,7.66,7.59,-5.40,0.99,2.14,3.64,-0.78
114,HAPV3,2021-03-01,17.20,17.29,16.24,16.32,15.74,15.50,5.29,12.47,20.75,2.32,-3.18
113,TRIS3,2021-02-26,9.96,9.99,9.19,9.36,9.96,9.91,-5.55,2.02,4.58,4.27,0.30
113,HBOR3,2021-02-26,8.63,8.73,8.06,8.07,8.42,8.72,-7.45,2.52,5.08,2.81,3.68
113,MEAL3,2021-02-26,3.30,3.30,3.10,3.10,3.28,3.25,-4.62,7.33,14.39,2.58,0.61


In [19]:
df[df.Date.dt.date == pd.to_datetime('today').date()].sort_values(by='var_vol',ascending=False)

,Date,Open,High,Low,Close,Volume,Acao,cotacao_hj,cotacao_dia_anterior,pct_do_dia,media_vol,var_vol,pct_longo
11,2021-03-05,15.740000,16.059999,15.710000,15.910000,0.3985,ELMD3,15.910000,17.500000,-9.085715,0.07970,inf,0.942801
118,2021-03-05,22.260000,23.219999,21.920000,22.920000,6.0570,RLOG3,22.920000,22.320000,2.688174,3.12548,4.339323,4.363002
118,2021-03-05,7.030000,7.110000,6.460000,6.530000,7.6570,WIZS3,6.530000,7.220000,-9.556781,3.90918,3.527759,8.882081
118,2021-03-05,12.870000,13.400000,11.960000,13.120000,3.5057,SOMA3,13.120000,13.000000,0.923076,1.53880,3.441550,8.841462
118,2021-03-05,7.550000,7.960000,7.550000,7.830000,3.8242,GRND3,7.830000,7.250000,7.999999,1.74668,3.046831,3.575986
84,2021-03-05,12.990000,13.190000,12.580000,12.790000,0.5262,TFCO4,12.790000,13.000000,-1.615385,0.29220,2.819029,3.127440
93,2021-03-05,7.810000,8.140000,7.390000,8.100000,9.7750,GMAT3,8.100000,7.800000,3.846156,4.98262,2.759195,8.765438
118,2021-03-05,22.000000,23.139999,21.250000,21.520000,4.1449,TUPY3,21.520000,23.090000,-6.799479,2.16796,2.688526,7.527876
118,2021-03-05,72.309998,79.839996,72.309998,78.230003,1.5968,ARZZ3,78.230003,73.000000,7.164388,0.88802,2.458582,7.567436
118,2021-03-05,13.120000,13.870000,12.960000,13.190000,8.1121,RAPT4,13.190000,13.010000,1.383546,4.57408,2.326597,1.743742


## Gráfico

In [ ]:
brf = yf.Ticker("BRFS3.SA").history(period='1y')
brf = brf.reset_index()

for i in ['Open','High','Close','Low']:
    brf[i] = brf[i].astype('float64')
    
fig = go.Figure(data=[go.Candlestick(x=brf['Date'],open=brf['Open'],high=brf['High'],low=brf['Low'],close=brf['Close'])])
#fig.show()

## Get .info

In [ ]:
ibr=pd.read_csv('./ibr.txt').tail(2)

df = pd.DataFrame(columns = ['Tick', 'profitMargins'])

for i, row in ibr.iterrows(): 
    acao = row.ticker+".SA"
    df.loc[i] = row.ticker, yf.Ticker(acao).info['profitMargins']

In [ ]:
yf.Ticker("VALE3.SA").history(period='max')